In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('upi_transactions_2024.csv')

In [4]:
df.head()

,transaction id,timestamp,transaction type,merchant_category,amount (INR),transaction_status,sender_age_group,receiver_age_group,sender_state,sender_bank,receiver_bank,device_type,network_type,fraud_flag,hour_of_day,day_of_week,is_weekend
0,TXN0000000001,2024-10-08 15:17:28,P2P,Entertainment,868,SUCCESS,26-35,18-25,Delhi,Axis,SBI,Android,4G,0,15,Tuesday,0
1,TXN0000000002,2024-04-11 06:56:00,P2M,Grocery,1011,SUCCESS,26-35,26-35,Uttar Pradesh,ICICI,Axis,iOS,4G,0,6,Thursday,0
2,TXN0000000003,2024-04-02 13:27:18,P2P,Grocery,477,SUCCESS,26-35,36-45,Karnataka,Yes Bank,PNB,Android,4G,0,13,Tuesday,0
3,TXN0000000004,2024-01-07 10:09:17,P2P,Fuel,2784,SUCCESS,26-35,26-35,Delhi,ICICI,PNB,Android,5G,0,10,Sunday,1
4,TXN0000000005,2024-01-23 19:04:23,P2P,Shopping,990,SUCCESS,26-35,18-25,Delhi,Axis,Yes Bank,iOS,WiFi,0,19,Tuesday,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   transaction id      250000 non-null  object
 1   timestamp           250000 non-null  object
 2   transaction type    250000 non-null  object
 3   merchant_category   250000 non-null  object
 4   amount (INR)        250000 non-null  int64 
 5   transaction_status  250000 non-null  object
 6   sender_age_group    250000 non-null  object
 7   receiver_age_group  250000 non-null  object
 8   sender_state        250000 non-null  object
 9   sender_bank         250000 non-null  object
 10  receiver_bank       250000 non-null  object
 11  device_type         250000 non-null  object
 12  network_type        250000 non-null  object
 13  fraud_flag          250000 non-null  int64 
 14  hour_of_day         250000 non-null  int64 
 15  day_of_week         250000 non-null  object
 16  is

In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [7]:
type(df['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
df.nunique()

transaction id        250000
timestamp             248610
transaction type           4
merchant_category         10
amount (INR)           10355
transaction_status         2
sender_age_group           5
receiver_age_group         5
sender_state              10
sender_bank                8
receiver_bank              8
device_type                3
network_type               4
fraud_flag                 2
hour_of_day               24
day_of_week                7
is_weekend                 2
dtype: int64

In [9]:
df.columns = [i.strip().lower().replace(' ','_') for i in df.columns]
print(df.columns)

Index(['transaction_id', 'timestamp', 'transaction_type', 'merchant_category',
       'amount_(inr)', 'transaction_status', 'sender_age_group',
       'receiver_age_group', 'sender_state', 'sender_bank', 'receiver_bank',
       'device_type', 'network_type', 'fraud_flag', 'hour_of_day',
       'day_of_week', 'is_weekend'],
      dtype='object')


In [10]:
df.rename(columns={'amount_(inr)': 'amount'}, inplace=True)

In [11]:
df['device_type'].unique()

array(['Android', 'iOS', 'Web'], dtype=object)

In [12]:
df.isnull().sum()

transaction_id        0
timestamp             0
transaction_type      0
merchant_category     0
amount                0
transaction_status    0
sender_age_group      0
receiver_age_group    0
sender_state          0
sender_bank           0
receiver_bank         0
device_type           0
network_type          0
fraud_flag            0
hour_of_day           0
day_of_week           0
is_weekend            0
dtype: int64

In [13]:
df.describe()

,timestamp,amount,fraud_flag,hour_of_day,is_weekend
count,250000,250000.000000,250000.000000,250000.000000,250000.000000
mean,2024-07-01 15:57:20.438195456,1311.756036,0.001920,14.681032,0.285348
min,2024-01-01 00:05:10,10.000000,0.000000,0.000000,0.000000
25%,2024-04-01 15:12:57.249999872,288.000000,0.000000,11.000000,0.000000
50%,2024-07-01 14:34:17,629.000000,0.000000,15.000000,0.000000
75%,2024-09-30 18:13:45,1596.000000,0.000000,19.000000,1.000000
max,2024-12-30 23:55:40,42099.000000,1.000000,23.000000,1.000000
std,NaN,1848.059224,0.043776,5.188304,0.451581


In [14]:
fraud_count = df[df['fraud_flag'] == 1].groupby('sender_age_group').size()
print(fraud_count)

sender_age_group
18-25    143
26-35    163
36-45    116
46-55     31
56+       27
dtype: int64


In [15]:
average_spent = df.groupby('merchant_category')['amount'].mean().sort_values(ascending=False)
print(average_spent)

merchant_category
Education        5094.017636
Shopping         2573.085398
Utilities        2361.110305
Fuel             1555.383434
Grocery          1166.350979
Other             848.777550
Healthcare        542.853905
Food              531.694480
Entertainment     413.325374
Transport         308.003780
Name: amount, dtype: float64


In [16]:
df.pivot_table(values='amount', 
               index='merchant_category', 
               columns='sender_age_group', 
               aggfunc='mean')

sender_age_group,18-25,26-35,36-45,46-55,56+
merchant_category,,,,,
Education,3097.595645,5287.210607,6716.587277,5900.342742,3865.399441
Entertainment,502.329744,440.259847,365.706309,301.153310,255.125743
Food,570.195075,560.031250,497.158234,469.533672,436.079787
Fuel,1387.726472,1531.052993,1777.435190,1674.260142,1204.511755
Grocery,951.784301,1177.441471,1341.554724,1253.892979,1101.726006
Healthcare,332.723813,471.735550,623.931291,778.438003,1193.427035
Other,920.624717,870.734209,817.673286,756.935444,669.449288
Shopping,3034.895198,2741.448533,2312.287671,1948.524446,1660.991962
Transport,340.700589,308.630527,303.398394,283.086767,210.213494


In [17]:
df.to_csv('upi_2024.csv',index=False)